<font color="#5642C2" size="5">Excersise 3 in Natural Language Processing</font>
<br>
<font color="#5642C2" size="4">Topic Modeling</font>
<br>
<font color="#5642C2" size="3">Author:</font><font color="#ADADAD" size="3">Angeliki Mylonaki</font>
<br>
<font color="#5642C2" size="3">Github:</font> https://github.com/geloumil/Topic-Modelling-and-NER.git

In [2]:
import glob
import os
import codecs

def getData(regex):

    file_list=[]
    trainFiles = glob.glob(os.path.join(".", regex)) #make list of paths

    for fd in trainFiles:
        with codecs.open(fd, encoding='utf-8', errors='ignore') as fp:
            file_list.append(fp.read().encode("utf-8"))
            
    return file_list

documents=getData("./nipstxt/nips*/*")
print type(documents[0])

<type 'str'>


<font color="#5642C2" size="3">Mining Documents</font>

In [56]:
from nltk import corpus
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import re



def mineDocs(documents):
    
    porter_stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    minedList=[]
    
    stopWords = corpus.stopwords.words("english")
    documents=[s.translate(None, string.punctuation) for s in documents]
    documents=[s.translate(None, "\x7f") for s in documents]   
    for doc in documents:
         
        #splitting document into sentences
        sentences = sent_tokenize(doc)

        for sentence in sentences:
            #removing digits
            sentence=sentence.translate(None, string.digits)
            
            #converting to lowercase
            sentence=sentence.lower()
            
            #removing stopwords
            sentence=' '.join([word for word in sentence.split() if word not in stopWords])
            
            #removing single letter words
            sentence=' '.join([word for word in sentence.split() if len(word) >1 ])
            
            #getting words of phrase
            words=word_tokenize(sentence)
            
            #stemming and lemmatizing
            words=[porter_stemmer.stem(word) for word in words]
            words=[wordnet_lemmatizer.lemmatize(word) for word in words]
                      
            minedList.append(sentence)
            
    return minedList



mdocs=mineDocs(documents)
mdocs_tokenized=[doc.split() for doc in mdocs]

<font color="#5642C2" size="3">Vectorizing,TF-IDF </font>

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(stop_words="english", binary="true")
m_docs_vector = count_vect.fit_transform(mdocs)

tfidf_transformer = TfidfTransformer(use_idf=False).fit(m_docs_vector)

<font color="#5642C2" size="3">Creating Document-term Matrix</font>

In [58]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary = corpora.Dictionary(mdocs_tokenized)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in mdocs_tokenized]

<font color="#5642C2" size="3">LDA</font>

In [59]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
from operator import itemgetter

num_topics=[10,50,20]
alpha=["auto","asymmetric","symmetric"]

# Running and Trainign LDA model on the document term matrix.
for topicNo in num_topics:
    for a in alpha:
        print "\n\nFor numberOfTopics=",topicNo," and a= ",a
        ldamodel = Lda(corpus=doc_term_matrix, num_topics=topicNo, id2word = dictionary,alpha=a)
        print ldamodel.print_topics(num_topics=topicNo, num_words=5)




For numberOfTopics= 10  and a=  auto
[(0, u'0.006*"set" + 0.006*"network" + 0.005*"learning" + 0.005*"neural" + 0.004*"networks"'), (1, u'0.005*"function" + 0.005*"one" + 0.005*"time" + 0.004*"neural" + 0.004*"model"'), (2, u'0.008*"learning" + 0.006*"data" + 0.006*"model" + 0.005*"network" + 0.005*"using"'), (3, u'0.006*"model" + 0.005*"neural" + 0.005*"network" + 0.004*"figure" + 0.004*"learning"'), (4, u'0.008*"learning" + 0.007*"network" + 0.007*"model" + 0.006*"figure" + 0.006*"neural"'), (5, u'0.007*"learning" + 0.006*"figure" + 0.005*"network" + 0.004*"function" + 0.004*"model"'), (6, u'0.008*"network" + 0.006*"learning" + 0.006*"model" + 0.005*"function" + 0.005*"neural"'), (7, u'0.006*"learning" + 0.006*"network" + 0.006*"function" + 0.005*"model" + 0.005*"neural"'), (8, u'0.008*"network" + 0.006*"input" + 0.006*"learning" + 0.005*"networks" + 0.005*"model"'), (9, u'0.010*"network" + 0.007*"neural" + 0.006*"model" + 0.006*"using" + 0.005*"learning"')]


For numberOfTopics= 1

[(0, u'0.007*"learning" + 0.005*"model" + 0.005*"neural" + 0.004*"network" + 0.004*"two"'), (1, u'0.006*"learning" + 0.005*"function" + 0.005*"network" + 0.005*"model" + 0.005*"used"'), (2, u'0.008*"model" + 0.006*"figure" + 0.006*"learning" + 0.005*"network" + 0.004*"algorithm"'), (3, u'0.007*"network" + 0.006*"function" + 0.006*"model" + 0.005*"learning" + 0.005*"output"'), (4, u'0.009*"network" + 0.007*"learning" + 0.006*"training" + 0.005*"function" + 0.005*"one"'), (5, u'0.008*"network" + 0.007*"model" + 0.006*"neural" + 0.005*"learning" + 0.005*"set"'), (6, u'0.007*"model" + 0.005*"neural" + 0.005*"network" + 0.005*"function" + 0.005*"figure"'), (7, u'0.008*"model" + 0.006*"neural" + 0.005*"figure" + 0.005*"learning" + 0.004*"network"'), (8, u'0.007*"learning" + 0.007*"model" + 0.006*"network" + 0.006*"function" + 0.005*"data"'), (9, u'0.007*"input" + 0.007*"network" + 0.005*"neural" + 0.005*"set" + 0.005*"time"'), (10, u'0.007*"model" + 0.005*"learning" + 0.005*"network" + 0.004

[(0, u'0.008*"network" + 0.008*"model" + 0.007*"learning" + 0.005*"function" + 0.005*"error"'), (1, u'0.006*"network" + 0.006*"model" + 0.005*"learning" + 0.005*"neural" + 0.004*"figure"'), (2, u'0.007*"network" + 0.006*"learning" + 0.005*"function" + 0.005*"neural" + 0.005*"figure"'), (3, u'0.008*"network" + 0.007*"learning" + 0.005*"input" + 0.005*"function" + 0.005*"model"'), (4, u'0.006*"data" + 0.006*"neural" + 0.005*"network" + 0.005*"learning" + 0.005*"input"'), (5, u'0.007*"model" + 0.006*"network" + 0.006*"learning" + 0.005*"data" + 0.005*"one"'), (6, u'0.006*"neural" + 0.006*"learning" + 0.006*"network" + 0.005*"figure" + 0.005*"input"'), (7, u'0.006*"network" + 0.006*"learning" + 0.006*"neural" + 0.005*"set" + 0.005*"training"'), (8, u'0.007*"network" + 0.005*"neural" + 0.005*"data" + 0.005*"set" + 0.005*"training"'), (9, u'0.008*"network" + 0.007*"neural" + 0.007*"input" + 0.006*"networks" + 0.005*"model"'), (10, u'0.007*"learning" + 0.007*"one" + 0.006*"model" + 0.005*"fig

<font color="#5642C2" size="3">Visualizing Results</font>

In [ ]:
#printing most representative topic for the 10 fist documents        
print "Best fitting topic for each Document"
print "------------------------------------"
for idx,doc in enumerate(doc_term_matrix[:5]):
    print "Document ",idx, "topic: ",\
    max(ldamodel.get_document_topics(doc),key=lambda item:item[1])[0]
print "......."

import pyLDAvis
import pyLDAvis.gensim

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

Best fitting topic for each Document
------------------------------------
Document  0 topic:  4
Document  1 topic:  12
Document  2 topic:  10
Document  3 topic:  9
Document  4 topic:  17
.......
